In [ ]:
# Quantum Artificial Neuron implementations based on papers:
# "An artificial neuron implemented on an actual quantum processor", Tacchino et al., 2019, and
# "Quantum computing model of an artificial neuron with continuously valued input data", Mangini et al., 2020.
# Using library Qiskit v. 0.26.0
# Author: Eduardo Barreto Brito/CIn UFPE

In [ ]:
# !pip install ipynb
# !pip install qiskit
# !pip install tabular
# !pip install itertools

In [1]:
import itertools
import matplotlib
import math
%matplotlib inline

In [2]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
from tabulate import tabulate

In [3]:
def find(word, character):
    return [i for i, ltr in enumerate(word) if ltr == character]

In [4]:
class QuantumArtificialNeuron():
    def __init__(self, qubit_amount):
        # We'll have [qubit_amount] qubits + 1 ancilla
        self._qubits_amount = qubit_amount
        self._quantum_registers = QuantumRegister(qubit_amount, 'q')
        self._ancilla = QuantumRegister(1, 'ancilla')
        
        # The classical register is only to be able to measure the response
        self._classical_register = ClassicalRegister(1, 'c')
        
        # Gathering the possible combinations between the QuBits (00, 01, 10, 11...)
        self._possible_qubits = self.get_possible_qubits(qubit_amount)
        
        #self._circuit = QuantumCircuit(self._quantum_registers, self._ancilla, self._classical_register)
        self._circuit = QuantumCircuit(self._quantum_registers)
        return
        
    def get_possible_qubits(self, qubit_amount):
        """ Gets all the possibilities for the amount of qubits. 
                Eg. for qubit_amount = 2, it will return ['00', '01', '10', '11'] """
        inputs = []
        for x in map(''.join, itertools.product('01', repeat=qubit_amount)):
            inputs.append(x)
        return inputs
    
    def append_hadamard_gates(self):
        """ Appends a barrier + Hadamard (h) gates to all QuBits """
        
        self._circuit.barrier()
        for qr in self._quantum_registers:
            self._circuit.h(qr)
        return

    def append_negative_gates_to_0_qubits(self, qubits):
        """ Appends negative (x) gates to all QuBits that are 0. 
                E.g. for qubit 100, x gates will be added on QuBit 1 and 2."""
        
        qr_num = 0
        for char_index in range(len(qubits)):
            qubit = qubits[char_index]
            if qubit == '0':
                self._circuit.x(self._quantum_registers[qr_num])
            qr_num += 1
        return
    
    def show_qubits_to_input_amplitudes_table(self, input):
        """ Prints a table with all QuBits to its respective amplitude.
            E.g. [1, -1, 1, -1]:
            00 -> 1
            01 -> -1
            10 -> 1
            11 -> -1"""
        headers = []
        index = 0
        for qr in self._quantum_registers:
            headers.append("q" + str(index))
            index += 1
        headers.append("ampl")
        
        qubits = []
        index = 0
        for entry in input:
            qubit = []
            qubits_string = self._possible_qubits[index]
            
            for char_index in range(len(qubits_string)):
                qubit.append(qubits_string[char_index])
            
            qubit.append(str(entry))
            qubits.append(qubit)
            
            index += 1
        
        print(tabulate(qubits, headers=headers))
        return
    
    def finish_circuit(self):
        """ Appends a CNot gate using all registers as controllers and ancilla as target, 
                and measure the ancilla to the classical register"""
        self._circuit.mcx(self._quantum_registers, self._ancilla)
        self._circuit.measure(self._ancilla, self._classical_register)
        return

In [5]:
class BruteForceQan(QuantumArtificialNeuron):
    def append_circuit(self, input):
        """ Appends the circuit responsible to mimic the input based on Tacchino et al. 2019 approach of Brute Force"""
        
        qr = self._quantum_registers
        circuit = self._circuit
        if len(input) > len(self._possible_qubits):
            raise Exception("The number of values of the input is bigger than the possible amount of values")
            
        index = 0
        for entry in input:
            if entry < 0:
                self._circuit.barrier()
                self.append_negative_gates_to_0_qubits(self._possible_qubits[index])
                
                # A multiple-controlled rotation z with lambda = pi is the same as having multiple-controlled z (e^(pi)i = 1)
                self._circuit.mcrz(math.pi, self._quantum_registers[:-1], self._quantum_registers[-1])
                
                self.append_negative_gates_to_0_qubits(self._possible_qubits[index])
            index += 1                
        return

In [6]:
class HSGSQan(QuantumArtificialNeuron):
    def reset_current_state(self):
        self._amplitude_state = [1] * len(self._possible_qubits)
        
    def update_current_amplitude_state(self, qubits_1s_indexes):
        """ Changes the amplitude signal of every QuBit that contains 1 on qubits_1s_indexes"""
        
        # When updating the amplitude current state, we need to switch the amplitude of every QuBit that the 
        # Z gate we just applied affects. That means that for every QuBit combination that all of the controlling QuBit is 1
        # we need to invert the amplitude, thus we need to iterate through all possible qubits
        index = 0
        for possible_qubit in self._possible_qubits:
            should_invert_amplitude = True
            
            # And then check where all the controlling QuBits are 1
            for qubit_1_index in qubits_1s_indexes:
                should_invert_amplitude = should_invert_amplitude and possible_qubit[qubit_1_index] == "1"
            
            # If all the controlling QuBits are one, we just invert the current amplitude
            if should_invert_amplitude:
                self._amplitude_state[index] = -self._amplitude_state[index]
                
            index += 1
        return
    
    def append_circuit(self, input):
        """ Appends the circuit responsible to mimic the input based on Tacchino et al. 2019 approach of HSGS 
        (hypergraph states generation subroutine) """
        
        qr = self._quantum_registers
        circuit = self._circuit
        self.reset_current_state()
        
        if len(input) > len(self._possible_qubits):
            raise Exception("The number of values of the input is bigger than the possible amount of values")
            
        iterating_possible_qubits = self._possible_qubits
        if input[0] < 0:
            self._circuit.barrier()
            self.append_negative_gates_to_0_qubits(self._possible_qubits[0])
            input = [element * (-1) for element in input]
            iterating_possible_qubits = self._possible_qubits[::-1]
        
        for num_of_1s in range(1, self._qubits_amount + 1):
            index = 0
            for possible_qubit in iterating_possible_qubits:
                # This means that we already have what we want, thus we can go back!
                if input == self._amplitude_state:
                    return
                
                # We found one difference. We must insert Z gates in the correct places
                elif input[index] != self._amplitude_state[index] and possible_qubit.count("1") == num_of_1s:
                    qubits_one = find(possible_qubit, "1")
                    control_qbits = []
                    for qubit_index in qubits_one:
                        control_qbits.append(self._quantum_registers[qubit_index])
                    
                    self._circuit.barrier()
                    
                    if len(control_qbits) == 1:
                        self._circuit.z(control_qbits[0])
                    else:
                        # A multiple-controlled rotation z with lambda = pi is the same as having multiple-controlled z (e^(pi)i = 1)
                        self._circuit.mcrz(math.pi, control_qbits[:-1], control_qbits[-1])
                    
                    # Now we can finally update our current state
                    self.update_current_amplitude_state(qubits_one)
                    
                index += 1            
        return

In [7]:
class PhaseShiftHSGSQan(QuantumArtificialNeuron):
    def append_circuit(self, input):
        qr = self._quantum_registers
        circuit = self._circuit
        if len(input) > len(self._possible_qubits):
            raise Exception("The number of values of the input is bigger than the possible amount of values")
            
        #TODO: Implement PhaseShiftQnn with HSGS
        # Most likely won't be implemented, since it was not part of neither Tacchino et al. [2019] nor Mangini et al. [2020]
        return

In [8]:
class PhaseShiftBruteForceQan(QuantumArtificialNeuron):
    def append_circuit(self, input):
        """ Appends the circuit responsible to mimic the input based on Mangini et al. 2020 approach of the continuously
            valued quantum neuron model"""
        qr = self._quantum_registers
        circuit = self._circuit
        if len(input) > len(self._possible_qubits):
            raise Exception("The number of values of the input is bigger than the possible amount of values")
            
        index = 0
        for entry in input:
            self._circuit.barrier()
            self.append_negative_gates_to_0_qubits(self._possible_qubits[index])

            # As we're having continuous valued entries, we can pass the entry already.
            if entry < 0:
                # We need to use Z gates only if we have negative amplitudes
                self._circuit.mcrz(entry, self._quantum_registers[:-1], self._quantum_registers[-1])
            else:
                # Otherwise, we can use normal PhaseGates
                self._circuit.mcp(entry, self._quantum_registers[:-1], self._quantum_registers[-1])

            self.append_negative_gates_to_0_qubits(self._possible_qubits[index])
            index += 1
        return